In [1]:
!pip install ast

ERROR: Could not find a version that satisfies the requirement ast (from versions: 0.0.1, 0.0.2)
ERROR: No matching distribution found for ast


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer



---

# Importing the datasets

---



In [55]:
df_links_small = pd.read_csv('/content/drive/MyDrive/DataSets/the-movies-dataset/links_small.csv')

In [56]:
df_keywords = pd.read_csv('/content/drive/MyDrive/DataSets/the-movies-dataset/keywords.csv')

In [57]:
df_credits = pd.read_csv('/content/drive/MyDrive/DataSets/the-movies-dataset/credits.csv')

In [58]:
df_movies_metadata_fixed = pd.read_csv('/content/drive/MyDrive/DataSets/the-movies-dataset/movies_metadata_fixed.csv')

In [59]:
display(df_links_small.head())

display(df_links_small.shape)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


(9125, 3)

In [60]:
display(df_keywords.head())

display(df_keywords.shape)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


(46419, 2)

In [61]:
display(df_credits.head())

display(df_credits.shape)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


(45476, 3)

In [62]:
display(df_movies_metadata_fixed.head(1))

display(df_movies_metadata_fixed.shape)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415


(45463, 24)



---
# Feature selection
---





In [63]:
df_movies_metadata_fixed = df_movies_metadata_fixed[['id', 'title', 'release_date', 'vote_count', 'vote_average', 'genres']]
df_movies_metadata_fixed.head()

,id,title,release_date,vote_count,vote_average,genres
0,862,Toy Story,1995-10-30,5415,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,1995-12-15,2413,6.9,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,1995-12-22,92,6.5,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,1995-12-22,34,6.1,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,1995-02-10,173,5.7,"[{'id': 35, 'name': 'Comedy'}]"


In [64]:
df = df_movies_metadata_fixed.merge(df_credits, how='inner', on='id')
df.head(1)

,id,title,release_date,vote_count,vote_average,genres,cast,crew
0,862,Toy Story,1995-10-30,5415,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."


In [65]:
df = df.merge(df_keywords, how='inner', on='id')
df.head()

,id,title,release_date,vote_count,vote_average,genres,cast,crew,keywords
0,862,Toy Story,1995-10-30,5415,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,1995-12-15,2413,6.9,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,1995-12-22,92,6.5,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,1995-12-22,34,6.1,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,Father of the Bride Part II,1995-02-10,173,5.7,"[{'id': 35, 'name': 'Comedy'}]","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [66]:
df.shape

(46628, 9)



---

# Work with categorical data

---



In [67]:
def handling_json(df, feature, value):
  df[feature] = df[feature].fillna('[]') \
                             .apply(ast.literal_eval) \
                             .apply(lambda x: [i[value] for i in x] if isinstance(x, list) else [])

  df[feature] = [', '.join(i) for i in df[feature]]
  df[feature] = pd.DataFrame(data=map(str.lower, df[feature])) 
  df[feature] = [i.replace(",", "") for i in df[feature]]

  return df

---
## Handling 'genres' column
---

In [68]:
def handling_genres_column_json(df):
  df['genres'] = df['genres'].fillna('[]') \
                             .apply(ast.literal_eval) \
                             .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

  '''mlb = MultiLabelBinarizer(sparse_output=True)

     df = df.join(pd.DataFrame.sparse.from_spmatrix(
                    mlb.fit_transform(df.pop('genres')),
                    index=df.index,
                    columns=mlb.classes_))'''

  df.genres = [', '.join(i) for i in df.genres]
  df.genres = pd.DataFrame(data=map(str.lower, df.genres)) 
  df.genres = [i.replace(",", "") for i in df.genres]

  return df

In [69]:
df = handling_genres_column_json(df)

In [81]:
df.genres.head()

0     animation comedy family
1    adventure fantasy family
2              romance comedy
3        comedy drama romance
4                      comedy
Name: genres, dtype: object



---

## Handling 'cast' column

---



In [71]:
def handling_cast_column_json(df):
  df['cast'] = df['cast'].fillna('[]') \
                             .apply(ast.literal_eval) \
                             .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

  df.cast = [', '.join(i) for i in df.cast]
  df.cast = pd.DataFrame(data=map(str.lower, df.cast)) 
  df.cast = [i.replace(" ", "") for i in df['cast']]
  df.cast = [i.replace(",", " ") for i in df['cast']]
  return df

In [72]:
df = handling_cast_column_json(df)

In [80]:
df.cast.head()

0    tomhanks timallen donrickles jimvarney wallace...
1    robinwilliams jonathanhyde kirstendunst bradle...
2    waltermatthau jacklemmon ann-margret sophialor...
3    whitneyhouston angelabassett lorettadevine lel...
4    stevemartin dianekeaton martinshort kimberlywi...
Name: cast, dtype: object



---

## Handling 'crew' column

---



In [75]:
df['crew'] = df['crew'].apply(ast.literal_eval)
df.crew.head()

0    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1    [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2    [{'credit_id': '52fe466a9251416c75077a89', 'de...
3    [{'credit_id': '52fe44779251416c91011acb', 'de...
4    [{'credit_id': '52fe44959251416c75039ed7', 'de...
Name: crew, dtype: object

In [76]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [79]:
df['director'] = df['crew'].apply(get_director)
df['director'] = df['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
df.director.head()

0      johnlasseter
1       joejohnston
2      howarddeutch
3    forestwhitaker
4      charlesshyer
Name: director, dtype: object